# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# Warm-up: numpy
## Principales variables
- a : float64, premier poids aléatoire du modèle

- b : float64, deuxième poids aléatoire du modèle

- c : float64, troisième poids aléatoire du modèle

- d : float64, quatrième poids aléatoire du modèle

- grad_a : float64, gradient de a par rapport à la loss (dL/da), calculé lors de la backward pass

- grad_b : float64, gradient de b par rapport à la loss (dL/db), calculé lors de la backward pass

- grad_c : float64, gradient de c par rapport à la loss (dL/dc), calculé lors de la backward pass

- grad_d : float64, gradient de d par rapport à la loss (dL/dd), calculé lors de la backward pass

- grad_y_pred : ndarray à 1 dimension de taille 2000, gradient de y par rapport à la loss, dérivée de la loss par rapport à la prédiction (dL/dy), calculé lors de la backward pass

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- loss : float64, résultat de la fonction de perte RMSE (root-mean-square error)

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : ndarray à 1 dimension de taille 2000, espace linéaire de -pi à pi, chaque point équidistant, variable indépendante et axe des abscisses

- y : ndarray à 1 dimension de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : ndarray à 1 dimension de taille 2000, prédiction du modèle pour la fonction sin(x)

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 908.1047510558714
199 637.7188666829088
299 448.91437027181234
399 316.98212874985825
499 224.7280110031763
599 160.1768496438894
699 114.98154627228752
799 83.319338647984
899 61.1253336275343
999 45.55976387039065
1099 34.63737108815822
1199 26.969357157834683
1299 21.583564957507612
1399 17.799073604855693
1499 15.138675271237926
1599 13.267745767988853
1699 11.951519513067268
1799 11.02520772651335
1899 10.373085697675682
1999 9.913848039186803
Result: y = -0.034042993771372126 + 0.8644133846217095 x + 0.005872982903423211 x^2 + -0.09442174126271155 x^3


# PyTorch: Tensors
## Principales variables
- a : torch.float tensor, rank 0, premier poids aléatoire du modèle, envoyé vers le device spécifié

- b : torch.float tensor, rank 0, deuxième poids aléatoire du modèle, envoyé vers le device spécifié

- c : torch.float tensor, rank 0, troisième poids aléatoire du modèle, envoyé vers le device spécifié

- d : torch.float tensor, rank 0, quatrième poids aléatoire du modèle, envoyé vers le device spécifié

- device : torch.device, device vers où envoyer les tensors (CPU ou GPU). De base, les tensors sont envoyés vers le
CPU, sauf si on décommente la ligne qui envoie vers le GPU (cuda:0). cuda:0 veut dire le GPU à l'indice lorsqu'on fait
la commande `nvidia-smi -L`.

- dtype : type de donnée spécifiée pour les tensors (torch.float)

- grad_a : torch.float tensor, rank 0, radient de a par rapport à la loss (dL/da), calculé lors de la backward pass

- grad_b : torch.float tensor, rank 0, gradient de b par rapport à la loss (dL/db), calculé lors de la backward pass

- grad_c : torch.float tensor, rank 0, gradient de c par rapport à la loss (dL/dc), calculé lors de la backward pass

- grad_d : torch.float tensor, rank 0, gradient de d par rapport à la loss (dL/dd), calculé lors de la backward pass

- grad_y_pred : torch.float tensor, rank 1, de taille 2000, gradient de y par rapport à la loss, dérivée de la loss par
rapport à la prédiction (dL/dy), calculé lors de la backward pass

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- loss : float, résultat de la fonction de perte RMSE (root-mean-square error)

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rank 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses, envoyé vers le device spécifié

- y : torch.float tensor, rank 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rank 1, de taille 2000, prédiction du modèle pour la fonction sin(x)


In [2]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 3711.523193359375
199 2482.61865234375
299 1662.5850830078125
399 1115.08544921875
499 749.3342895507812
599 504.8507385253906
699 341.3248291015625
799 231.8761749267578
899 158.5711669921875
999 109.43869018554688
1099 76.48291015625
1199 54.36067199707031
1299 39.498680114746094
1399 29.50576400756836
1499 22.78086280822754
1599 18.251150131225586
1699 15.197304725646973
1799 13.136401176452637
1899 11.74432373046875
1999 10.803014755249023
Result: y = 0.029310109093785286 + 0.8228000998497009 x + -0.005056481342762709 x^2 + -0.08850260823965073 x^3


# PyTorch: Tensors and autograd
## Description

In [3]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 464.5907287597656
199 330.585693359375
299 236.01516723632812
399 169.26226806640625
499 122.13695526123047
599 88.86296844482422
699 65.36573791503906
799 48.77046203613281
899 37.048248291015625
999 28.767196655273438
1099 22.91651725769043
1199 18.78248405456543
1299 15.861126899719238
1399 13.796542167663574
1499 12.337331771850586
1599 11.305903434753418
1699 10.576793670654297
1799 10.061360359191895
1899 9.696952819824219
1999 9.439310073852539
Result: y = -0.026266925036907196 + 0.8542943596839905 x + 0.004531480371952057 x^2 + -0.09298239648342133 x^3


# PyTorch: Defining new autograd functions
## Description


In [4]:
# -*- coding: utf-8 -*-
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')


99 209.9583282470703
199 144.6602020263672
299 100.7025146484375
399 71.03520965576172
499 50.97850799560547
599 37.40315246582031
699 28.20688247680664
799 21.97319221496582
899 17.7457275390625
999 14.877889633178711
1099 12.93176555633545
1199 11.610918998718262
1299 10.71424674987793
1399 10.105476379394531
1499 9.69210433959961
1599 9.411375045776367
1699 9.220744132995605
1799 9.091285705566406
1899 9.003360748291016
1999 8.943641662597656
Result: y = 3.5881797533221516e-09 + -2.208526849746704 * P3(-1.6777875755380478e-09 + 0.2554861009120941 x)


# PyTorch: nn
## Description


In [5]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1246.935791015625
199 828.8331909179688
299 551.97998046875
399 368.6396179199219
499 247.21412658691406
599 166.78565979003906
699 113.50628662109375
799 78.20741271972656
899 54.817779541015625
999 39.317413330078125
1099 29.043764114379883
1199 22.233261108398438
1299 17.717775344848633
1399 14.723373413085938
1499 12.737268447875977
1599 11.419677734375
1699 10.54539966583252
1799 9.965130805969238
1899 9.57993221282959
1999 9.324136734008789
Result: y = 0.006657042074948549 + 0.8357245326042175 x + -0.0011484501883387566 x^2 + -0.0903410017490387 x^3


# PyTorch: optim
## Description


In [6]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')


99 1917.0157470703125
199 1262.92919921875
299 965.7806396484375
399 746.182373046875
499 566.9778442382812
599 422.24993896484375
699 306.98193359375
799 216.60821533203125
899 146.58819580078125
999 93.43482971191406
1099 55.207584381103516
1199 30.27956199645996
1299 16.435304641723633
1399 10.59294319152832
1499 9.07484245300293
1599 8.904926300048828
1699 8.903571128845215
1799 8.949064254760742
1899 8.939556121826172
1999 8.932462692260742
Result: y = 0.0006567542441189289 + 0.8572276830673218 x + 0.0006570336408913136 x^2 + -0.09282868355512619 x^3


# PyTorch: Custom nn Modules
## Description


In [7]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 4830.9033203125
199 3197.523681640625
299 2117.42333984375
399 1403.1854248046875
499 930.880126953125
599 618.5574340820312
699 412.0263977050781
799 275.4522705078125
899 185.13888549804688
999 125.41643524169922
1099 85.92308807373047
1199 59.806671142578125
1299 42.5363655090332
1399 31.11566734313965
1499 23.563196182250977
1599 18.568819046020508
1699 15.266027450561523
1799 13.081899642944336
1899 11.63752269744873
1999 10.682350158691406
Result: y = -0.001859007403254509 + 0.8147933483123779 x + 0.00032071140594780445 x^2 + -0.08736371994018555 x^3


# PyTorch: Control Flow + Weight Sharing
## Description


In [8]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 137.28123474121094
3999 66.01779174804688
5999 36.05306625366211
7999 20.942001342773438
9999 14.36691951751709
11999 11.358105659484863
13999 9.890912055969238
15999 9.349603652954102
17999 8.933727264404297
19999 8.738455772399902
21999 8.903573989868164
23999 8.634082794189453
25999 8.844919204711914
27999 8.85088062286377
29999 8.862196922302246
Result: y = -0.0004927232512272894 + 0.8564959168434143 x + -0.00044520897790789604 x^2 + -0.09365628659725189 x^3 + 0.00011815274774562567 x^4 ? + 0.00011815274774562567 x^5 ?
